<a href="https://colab.research.google.com/github/BhojRajBist/BhojRajBist/blob/main/LST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install rasterio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 39.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from google.colab import drive
import requests


In [ ]:
file_path = f'{data_path}/Landsat8_image.tif'  # Replace with your file name

with rasterio.open(file_path) as src:
    band_red = src.read(4)  # Band 4 (Red)
    band_nir = src.read(5)  # Band 5 (NIR)
    band_swir = src.read(6)  # Band 6 (SWIR)
    band_tir = src.read(10)  # Band 10 (Thermal)

# Plot the bands
fig, ax = plt.subplots(1, 4, figsize=(20, 5))
ax[0].imshow(band_red, cmap='Reds')
ax[0].set_title('Red Band')
ax[1].imshow(band_nir, cmap='Greens')
ax[1].set_title('NIR Band')
ax[2].imshow(band_swir, cmap='Blues')
ax[2].set_title('SWIR Band')
ax[3].imshow(band_tir, cmap='hot')
ax[3].set_title('Thermal Band')
plt.show()


NameError: name 'data_path' is not defined

In [ ]:
# Constants for Landsat 8
ML = 0.0003342  # Band specific multiplicative rescaling factor
AL = 0.1  # Band specific additive rescaling factor
K1 = 774.89  # Calibration constant 1
K2 = 1321.08  # Calibration constant 2

# Convert DN to radiance
radiance = ML * band_tir + AL

# Convert radiance to brightness temperature
brightness_temp = K2 / np.log((K1 / radiance) + 1) - 273.15

# Calculate NDVI for emissivity calculation
ndvi = (band_nir - band_red) / (band_nir + band_red)

# Calculate emissivity
emissivity = 0.004 * ndvi + 0.986

# Calculate LST
lst = brightness_temp / (1 + (10.8 * brightness_temp / 14388) * np.log(emissivity))

plt.imshow(lst, cmap='hot')
plt.colorbar(label='LST (°C)')
plt.title('Land Surface Temperature')
plt.show()


In [ ]:
# Example: Load and process LST for different years, then plot trends
# Repeat the above steps for multiple years and store LST values

years = ['2004', '2009', '2014', '2019', '2024']
lst_values = []  # Store LST values for different years

for year in years:
    file_path = f'{data_path}/Landsat_{year}.tif'
    with rasterio.open(file_path) as src:
        band_tir = src.read(10)
        radiance = ML * band_tir + AL
        brightness_temp = K2 / np.log((K1 / radiance) + 1) - 273.15
        lst = brightness_temp / (1 + (10.8 * brightness_temp / 14388) * np.log(emissivity))
        lst_values.append(np.mean(lst))

plt.plot(years, lst_values, marker='o')
plt.xlabel('Year')
plt.ylabel('Average LST (°C)')
plt.title('LST Trends Over Years')
plt.show()


NameError: name 'data_path' is not defined

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from google.colab import drive
import requests


In [ ]:
drive.mount('/content/sam')
# Replace 'path_to_your_data' with the path to your Landsat data
data_path = '/content/sam'


Drive already mounted at /content/sam; to attempt to forcibly remount, call drive.mount("/content/sam", force_remount=True).


In [ ]:
!pip install earthengine-api geemap



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.9 MB/s eta 0:00:00


In [2]:
import ee
import geemap

# Authenticate your Google Earth Engine account
ee.Authenticate()

# Initialize the Earth Engine API
# ee.Initialize()


True

In [3]:
# Initialize the Earth Engine API
ee.Initialize()

EEException: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.

In [ ]:
# Define a region of interest (ROI)
roi = ee.Geometry.Polygon(
    [[[85.3240, 27.7172],
      [85.3240, 27.6240],
      [85.5140, 27.6240],
      [85.5140, 27.7172]]]
)

# Define the time range
start_date = '2023-01-01'
end_date = '2023-12-31'

# Filter Sentinel-2 data
sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate(start_date, end_date) \
    .filterBounds(roi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median() \
    .clip(roi)

# Visualize the image
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}
Map = geemap.Map()
Map.centerObject(roi, 10)
Map.addLayer(sentinel2, vis_params, 'Sentinel-2 Image')
Map
